In [1]:
import re
import pandas as pd
import numpy as np
import os
import datetime

In [37]:
date_list =[]
mae_list = []
smape_list = []
with open('E:\day-ahead price\Code\Time-Series-Library-main\Results\iso tmp/result_long_term_forecast.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()  # 读取所有行
    for i in range(len(lines)):
        line = lines[i].strip()  # 去除文本中的换行符
        index = line.find('2023-')  # 查找'2023-'在每一行中的位置
        index_dataset = line.find('iso')
        if index_dataset != -1:
            if index != -1:  # 如果找到了'2023-'
                result = line[index:index+10]  # 记录'2023-'及其后面的5个字符
                date_list.append(pd.to_datetime(result))
                if i+1 < len(lines):  # 如果存在下一行
                    next_line = lines[i+1].strip()  # 获取下一行内容
                    numbers = re.findall(r'\d+\.\d+|\d+', next_line)  # 使用正则表达式找出所有的数字，包括整数和小数
                    float_numbers = [float(num) for num in numbers]  # 将字符串转换为浮点数
                    mae_list.append(float_numbers[1])
                    smape_list.append(float_numbers[2])
df_result = pd.DataFrame({'date': date_list, 'mae': mae_list, 'smape': smape_list})
df_result

,date,mae,smape
0,2023-01-01,20.540403,0.422588
1,2023-01-02,23.708298,0.543241
2,2023-01-03,16.477646,0.359004
3,2023-01-04,11.551124,0.225557
4,2023-01-05,17.482805,0.356882
...,...,...,...
268,2023-09-26,14.453778,0.491869
269,2023-09-27,12.912598,0.406728
270,2023-09-28,15.618917,0.516831
271,2023-09-29,12.696010,0.408935


In [38]:
df_result['mae'].mean(), df_result['smape'].mean()

(23.535681303604182, 0.5375001598627139)

In [34]:
df_month = df_result[(pd.to_datetime('2023-04-01') <= df_result['date']) & (df_result['date'] <= pd.to_datetime('2023-06-25'))]
df_month['mae'].mean(), df_month['smape'].mean()

(25.155232778815336, 0.6642641980460909)

capacity prediction

In [28]:
path = r'E:\day-ahead price\Code\Time-Series-Library-main\Results\capacity timesnet\results'
os.chdir(path)
files = os.listdir(path)
df = pd.DataFrame()
df_d3 = pd.DataFrame()
df_d4 = pd.DataFrame()
df_d5 = pd.DataFrame()
for file in files:
    # print(file)
    # find date in each file name
    index = file.find('2023-')
    date = file[index:index+10]
    # change directory to each file
    os.chdir(path + '\\' + file)
    # read each npy file
    data_pred = np.load('pred.npy', allow_pickle=True)
    data_true = np.load('true.npy', allow_pickle=True)
    # data_metric = np.load('metrics.npy', allow_pickle=True)
    # print(data_pred, data_true)
    data_true_d2 = data_true.reshape(-1)[1]
    data_pred_d2 = data_pred.reshape(-1)[1]
    data_pred_d3 = data_pred.reshape(-1)[2]
    data_pred_d4 = data_pred.reshape(-1)[3]
    data_pred_d5 = data_pred.reshape(-1)[4]
    # print(data_true_d2, data_pred_d2)
    # calculate mae and smape
    mae = np.mean(np.abs(data_true_d2 - data_pred_d2))
    smape = np.mean(np.abs(data_true_d2 - data_pred_d2) / ((np.abs(data_true_d2) + np.abs(data_pred_d2)) / 2))
    # print(mae, smape)
    df = df.append({'date': pd.to_datetime(date) - datetime.timedelta(3), 'mae': mae, 'smape': smape, 'pred': data_pred_d2, 'true': data_true_d2}, ignore_index=True)
    df_d3 = df_d3.append({'date': pd.to_datetime(date) - datetime.timedelta(2), 'pred': data_pred_d3}, ignore_index=True)
    df_d4 = df_d4.append({'date': pd.to_datetime(date) - datetime.timedelta(1), 'pred': data_pred_d4}, ignore_index=True)
    df_d5 = df_d5.append({'date': pd.to_datetime(date), 'pred': data_pred_d5}, ignore_index=True)
df
    

C:\Users\PC\AppData\Local\Temp\ipykernel_10268\3933643260.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'date': pd.to_datetime(date) - datetime.timedelta(3), 'mae': mae, 'smape': smape, 'pred': data_pred_d2, 'true': data_true_d2}, ignore_index=True)
C:\Users\PC\AppData\Local\Temp\ipykernel_10268\3933643260.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_d3 = df_d3.append({'date': pd.to_datetime(date) - datetime.timedelta(2), 'pred': data_pred_d3}, ignore_index=True)
C:\Users\PC\AppData\Local\Temp\ipykernel_10268\3933643260.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_d4 = df_d4.append({'date': pd.to_datetime(date) - datetime.timedelta(1), 'pred': data_pred_d4}, ignore_index=True)
C:

,date,mae,smape,pred,true
0,2023-03-29,284.753906,0.006752,42315.753906,42031.0
1,2023-03-30,3192.089844,0.078147,42443.089844,39251.0
2,2023-03-31,4992.808594,0.124517,42593.808594,37601.0
3,2023-04-01,9342.160156,0.250951,41898.160156,32556.0
4,2023-04-02,9776.203125,0.265917,41652.203125,31876.0
...,...,...,...,...,...
270,2023-12-24,236.324219,0.004821,49142.324219,48906.0
271,2023-12-25,236.328125,0.004821,49142.328125,48906.0
272,2023-12-26,236.324219,0.004821,49142.324219,48906.0
273,2023-12-27,236.324219,0.004821,49142.324219,48906.0


In [30]:
df_month = df[(pd.to_datetime('2023-03-01') <= df['date']) & (df['date'] <= pd.to_datetime('2023-12-31'))]
df_month['mae'].mean(), df_month['smape'].mean()

(1863.7028, 0.04832246603378979)

In [31]:
df.to_csv('E:\day-ahead price\Code\Time-Series-Library-main\Results\capacity timesnet\capacity_timesnet.csv', index=False)
df_d3.to_csv('E:\day-ahead price\Code\Time-Series-Library-main\Results\capacity timesnet\capacity_timesnet_d3.csv', index=False)
df_d4.to_csv('E:\day-ahead price\Code\Time-Series-Library-main\Results\capacity timesnet\capacity_timesnet_d4.csv', index=False)
df_d5.to_csv('E:\day-ahead price\Code\Time-Series-Library-main\Results\capacity timesnet\capacity_timesnet_d5.csv', index=False)